In [1]:
import pandas as pd
import sqlite3

# Load your sessions CSV
df = pd.read_csv("/content/online_shoppers_intention.csv")  # Replace with your file path

# Connect to SQLite in memory
conn = sqlite3.connect(':memory:')
df.to_sql('sessions', conn, index=False, if_exists='replace')

# SQL queries dictionary
session_queries = {
    "Total number of sessions": "SELECT COUNT(*) AS TotalSessions FROM sessions;",
    "Distinct Visitor Types": "SELECT DISTINCT VisitorType FROM sessions;",
    "Total sessions by VisitorType": """
        SELECT VisitorType, COUNT(*) AS TotalSessions
        FROM sessions
        GROUP BY VisitorType
        ORDER BY TotalSessions DESC;
    """,
    "Average PageValues for buyers vs non-buyers": """
        SELECT Revenue, ROUND(AVG(PageValues), 2) AS AvgPageValue
        FROM sessions
        GROUP BY Revenue;
    """,
    "BounceRates difference between Weekend and Weekday traffic": """
        SELECT Weekend, ROUND(AVG(BounceRates), 3) AS AvgBounceRate
        FROM sessions
        GROUP BY Weekend;
    """,
    "Most common month for purchases": """
        SELECT Month, COUNT(*) AS PurchaseCount
        FROM sessions
        WHERE Revenue = 1
        GROUP BY Month
        ORDER BY PurchaseCount DESC;
    """,
    "Average Administrative Duration by month": """
        SELECT Month, ROUND(AVG(Administrative_Duration), 2) AS AvgAdminDuration
        FROM sessions
        GROUP BY Month
        ORDER BY AvgAdminDuration DESC;
    """,
    "Compare ExitRates for buyers vs non-buyers": """
        SELECT Revenue, ROUND(AVG(ExitRates), 3) AS AvgExitRate
        FROM sessions
        GROUP BY Revenue;
    """,
    "VisitorType with highest conversion rate": """
        SELECT VisitorType,
               ROUND(SUM(CASE WHEN Revenue = 1 THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) AS ConversionRate
        FROM sessions
        GROUP BY VisitorType
        ORDER BY ConversionRate DESC;
    """,
    "Correlation proxy: PageValues vs Purchases": """
        SELECT CASE
                   WHEN PageValues = 0 THEN 'No PageValue'
                   WHEN PageValues BETWEEN 0 AND 100 THEN 'Low (0-100)'
                   WHEN PageValues BETWEEN 101 AND 500 THEN 'Medium (101-500)'
                   ELSE 'High (500+)'
               END AS PageValueRange,
               COUNT(*) AS SessionCount,
               SUM(CASE WHEN Revenue = 1 THEN 1 ELSE 0 END) AS Purchases
        FROM sessions
        GROUP BY PageValueRange
        ORDER BY Purchases DESC;
    """,
    "Top 5 months with highest conversion rates": """
        SELECT Month,
               ROUND(SUM(CASE WHEN Revenue = 1 THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) AS ConversionRate
        FROM sessions
        GROUP BY Month
        ORDER BY ConversionRate DESC
        LIMIT 5;
    """
}

# Execute and display all queries
for name, query in session_queries.items():
    print(f"\n===== {name} =====\n")
    display(pd.read_sql_query(query, conn))



===== Total number of sessions =====



,TotalSessions
0,12205



===== Distinct Visitor Types =====



,VisitorType
0,Returning_Visitor
1,New_Visitor
2,Other



===== Total sessions by VisitorType =====



...

,VisitorType,ConversionRate
0,New_Visitor,24.93
1,Other,19.75
2,Returning_Visitor,14.09



===== Correlation proxy: PageValues vs Purchases =====



,PageValueRange,SessionCount,Purchases
0,Low (0-100),2648,1467
1,No PageValue,9475,370
2,Medium (101-500),78,67
3,High (500+),4,4



===== Top 5 months with highest conversion rates =====



,Month,ConversionRate
0,Nov,25.49
1,Oct,20.95
2,Sep,19.20
3,Aug,17.55
4,Jul,15.28
